In [1]:
# STEP 1: Install specific versions of numpy and datasets
!pip uninstall -y numpy datasets
!pip install --force-reinstall numpy==1.26.4 datasets==2.14.5


Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: datasets 2.14.5
Uninstalling datasets-2.14.5:
  Successfully uninstalled datasets-2.14.5
  Using cached numpy-1.26.4-cp39-cp39-win_amd64.whl.metadata (61 kB)
  Using cached datasets-2.14.5-py3-none-any.whl.metadata (19 kB)
  Using cached pyarrow-20.0.0-cp39-cp39-win_amd64.whl.metadata (3.4 kB)
  Using cached dill-0.3.7-py3-none-any.whl.metadata (9.9 kB)
  Using cached pandas-2.2.3-cp39-cp39-win_amd64.whl.metadata (19 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached xxhash-3.5.0-cp39-cp39-win_amd64.whl.metadata (13 kB)
  Using cached multiprocess-0.70.18-py39-none-any.whl.metadata (7.5 kB)
  Using cached fsspec-2023.6.0-py3-none-any.whl.metadata (6.7 kB)
  Using cached aiohttp-3.12.0-cp39-cp39-win_amd64.whl.metadata (7.9 kB)
  Using cached huggingface_hub-

In [2]:
# STEP 2: Load the SST-2 dataset from the GLUE benchmark
from datasets import load_dataset
dataset = load_dataset("glue", "sst2")
print(dataset["train"][0])


c:\Users\ragha\anaconda3\envs\Python9\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'sentence': 'hide new secretions from the parental units ', 'label': 0, 'idx': 0}


In [3]:
# STEP 3: Load the BERT tokenizer
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


In [4]:
# STEP 4: Tokenize the dataset
def tokenize_function(example):
    return tokenizer(example["sentence"], padding="max_length", truncation=True, max_length=128)

# Apply the tokenizer to the dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Remove unnecessary columns and format the dataset
tokenized_dataset = tokenized_dataset.remove_columns(["sentence", "idx"])
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


Map: 100%|██████████| 1821/1821 [00:00<00:00, 15377.55 examples/s]


In [5]:
# STEP 5: Load the BERT model for sequence classification
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# STEP 6: Set up the Trainer and evaluation metrics
import os
os.environ["WANDB_DISABLED"] = "true"  # Disable Weights & Biases logging

from transformers import TrainingArguments, Trainer
import numpy as np
from sklearn.metrics import accuracy_score

# Define the compute_metrics function for evaluation
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, predictions)}

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    do_eval=True,
    save_steps=500,
    eval_steps=500,
    report_to="none"
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    compute_metrics=compute_metrics,
)


In [7]:
# STEP 7: Start training
trainer.train()


Step,Training Loss
10,0.680300
20,0.645800
30,0.556900
40,0.466700
50,0.453000
60,0.361300
70,0.340200
80,0.541900
90,0.357400
100,0.386500


KeyboardInterrupt: 